In [27]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio import Entrez
from Bio.Align.Applications import MafftCommandline
import re

In [28]:

seq1 = "ATGCCATGCAAA"
seq2 = "ATTGATCATTT"
def creaMat (seq1,seq2) :
    #Creation de la matrice initiale :
    maxligne = len(seq1)+1
    maxcolone = len(seq2)+1
    matrix = [[0 for j in range(maxcolone)] for i in range(maxligne)]
    #Remplissage de la matrice initiale
    cpt = 0
    for i in range(maxligne) :
        matrix[i][0] = cpt 
        cpt-= 1

    cpt = 0
    for j in range(maxcolone) :
        matrix[0][j] = cpt 
        cpt-= 1
    
    return(matrix)
    

In [29]:
# Remplissage de la matrice :
    
def petitChemin (seq1,seq2,mat,li,col,penalite):
    valDiag = 0
    valDroite = 0
    valGauche = 0
    #Quand la matrice est a 0 on retrouve la premiere ligne d'initialisation avant de pouvoir comparer les sequences 
    if li == 0 and col == 0:
        if seq1[li] == seq2[col] :
            valDiag = 2
        else :
            valDiag = -penalite
        return(valDiag,li+1,col+1)
    #Les lignes et colone de la matrice ont 1 de plus que les sequences car les seq commencent a 0
    else :
        if seq1[li] == seq2[col-1]:
            valDroite = mat[li][col]+2
        else :
            valDroite = mat[li][col] -penalite

        if seq1[li-1] == seq2[col] :
            valGauche = mat[li][col]+2
        else :
            valGauche = mat[li][col] -penalite

        if seq1[li] == seq2[col] :
            valDiag = mat[li][col]+2
        else :

            valDiag = mat[li][col] -penalite

    #Sinon on compare les valeurs d'avant pour eviter les out of range
    if max(valDiag,valDroite,valGauche) == valGauche :
        return(valGauche,li,col+1)
    elif max(valDiag,valDroite,valGauche) == valDroite :
        return(valDroite,li+1,col)
    elif max(valDiag,valDroite,valGauche) == valDiag :
        return(valDiag,li+1,col+1)
    else :
        idemDiag = petitChemin (seq1,seq2,matrix,ligne+1,colone+1)
        idemDroit =petitChemin (seq1,seq2,matrix,ligne,colone+1)
        idemGauche =petitChemin (seq1,seq2,matrix,ligne+1,colone)
        if max(idemDiag[0],idemDroit[0],idemGauche[0]) == idemDiag :
            return (valDiag,li+1,col+1,idemDiag)
        elif max(idemDiag[0],idemDroit[0],idemGauche[0]) == idemDroite :
            return(valDroite,li+1,col,idemDroit)
        else :
            return(valGauche,li,col+1,idemGauche)

def trouvePetitChemin (seq1,seq2,penal) :
    matrix = creaMat(seq1,seq2)
    traceback = creaMat(seq1,seq2)
    ligne = 0
    colone = 0
    while colone < len(seq2) and ligne < len(seq1) :
            res = petitChemin(seq1,seq2,matrix,ligne,colone,penal)
            ligne = res[1]
            colone = res[2]
            matrix[ligne][colone] = res[0]
            traceback[ligne][colone] = 1
    if ligne == len(seq1):
        print("ligne OK")
    else :
        while ligne < len(seq1) :
            lignedep = ligne
            colonedep = colone
            ligne+=1
            if seq1[ligne-1] == seq2[colone-1] and ligne-1 == lignedep:
                matrix[ligne][colone] = matrix[lignedep][colonedep]+2
                traceback[ligne][colone] = 1
            else :
                matrix[ligne][colone] = matrix[lignedep][colonedep]-1
                traceback[ligne][colone] = 1            
            if seq1[ligne-1] == seq2[colone-1] :
                matrix[ligne][colone] = matrix[ligne-1][colone]+2
                traceback[ligne][colone] = 1
            else :
                matrix[ligne][colone] = matrix[ligne-1][colone]-1
                traceback[ligne][colone] = 1
        # si on arrive sur un des bord avant d'arriver au score optimal 
        while colone < len(seq2) :
            colonedep = colone
            lignedep = ligne
            colone+=1
            if seq1[ligne-1] == seq2[colone-1] and colone-1 == colonedep:
                matrix[ligne][colone] = matrix[lignedep][colonedep]+2
                traceback[ligne][colone] = 1
            else :
                matrix[ligne][colone-1] = matrix[lignedep][colonedep]-1
                traceback[ligne][colone] = 1
            
            if seq1[ligne-1] == seq2[colone-1] :
                matrix[ligne][colone] = matrix[ligne][colone-1]+2
                traceback[ligne][colone] = 1
            else :
                matrix[ligne][colone] = matrix[ligne][colone-1]-1
                traceback[ligne][colone] = 1
            
        score = matrix[ligne][colone]
            
    return(traceback,score)





In [30]:
def afficheMat (seq1,seq2,mat) :
    ligne = 0
    cpt = 0
    nbnucl = 0
    cptligne = 0
    for nucl in seq2 :
        if nbnucl != 0 :
            print(" "+str(nucl), end ="")
        else :
            print("     "+str(nucl),end = "")
        nbnucl+=1
    print("")
    for liste in mat :
        if cpt >= 1 :
            print("")
            print(str(seq1[ligne-1])+" ",end = "")
        for num in liste:
            if cptligne == 0:
                print("   "+str(num),end = "")
            else :
                print(str(num)+" ",end = "")
            cptligne +=1
        ligne+=1
        cpt += 1
    return 
    


In [31]:

def aligne (seq1,seq2,traceback) :
    ligne = len(seq1)
    colone = len(seq2)
    cptligne = 0
    cptcol = 0
    while ligne != 1 and colone != 1 :
        if traceback[ligne-1][colone] == 1 :
            seq2 = seq2[:colone+cptcol]+"-"+seq2[colone+cptcol:]
            ligne-=1
            cptcol += 1
            cptligne = 0
            

        elif traceback[ligne][colone-1] == 1 :
            seq1 = seq1[:ligne+cptligne]+"-"+seq1[ligne+cptligne:]
            colone -= 1
            cptligne += 1
            cptcol = 0
        else :
            ligne -=1
            colone -=1

    return(seq1,seq2)
        


In [32]:
#creation du fichier de retour des sequences aligné
def creaFasta (resultat,score):
    with open( "sequence_aligne.fasta", "w") as w :
        w.write(">sequence 1 \n")
        w.write(str(resultat[0])+"\n")
        w.write(">sequence 2 \n")
        w.write(str(resultat[1])+"\n")
        w.write("Le score d'alignement est de : "+str(score))

In [36]:
# Mise en place d'une commande unique 
def monAlignement (seq1,seq2,penalite = 1,fasta = False) :
    # Verification de la validité des argument rentré :
    expCarac = "^[a-zA-Z]+$"
    if not(re.match(expCarac,seq1)) :
        print("La premiere sequence entree n'est pas valide")
    else :
        sequence1 = seq1

    if not(re.match(expCarac,seq2)) :
        print("La seconde sequence entree n'est pas valide")
    else :
        sequence2 = seq2

    expPenal = "^[+]?([1-9]\d*(\.\d+)?|0\.\d*[1-9])$"
    if not(re.match(expPenal,str(penalite))) :
        print("La penalite rentree n'est pas un nombre superieur a 0")
        return
    
    #Creation de la matrice avec penalité :
    score = trouvePetitChemin(sequence1,sequence2,penalite)[1]
    traceback = trouvePetitChemin(sequence1,sequence2,penalite)[0]

    #Alignement des deux sequences  :
    res = aligne(sequence1,sequence2,traceback)

    #Creation du fichier fasta avec les deux sequences :
    if fasta == True :    
        creaFasta(res,score)   
        print("Le fichier fasta creer a pour nom : sequence_aligne.fasta \n")
        return "Voir fasta"
    else :
        return (res,score)  # Retours des sequences aligne si elle ne sont pas retourner dans un fasta 
        

In [37]:
#Test de la fonction
print(monAlignement(seq1,seq2))
monAlignement(seq1,seq2,fasta = True)
print(monAlignement(seq1,seq2,penalite = 4))
print(monAlignement(seq1,seq2,penalite = -2))

(('ATG---CCAT--GCAAA', 'ATTGATC-ATTT-----'), 13)
Le fichier fasta creer a pour nom : sequence_aligne.fasta 

(('ATG---CCAT--GCAAA', 'ATTGATC-ATTT-----'), 7)
La penalite rentree n'est pas un nombre superieur a 0
None
